# **XGBoost**
- Using data that was annotated
- Training it on the unbalanced data
- Using avgpool
- 1 second window
- Entropy is the measure of information contained in a state
- Testing it on the best perfoming combiantion of preprocessing and features



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install xgboost

In [3]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For audio
from IPython.display import Audio
import librosa

# For preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# For modeling
import xgboost as xgb
from sklearn.metrics import classification_report, balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import scipy.ndimage
import pygame
import time
from scipy.signal import butter, filtfilt
import random

pygame 2.6.0 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
pkl_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/feature-extraction/Annotated/Regular/AveragePooled/split_features_3s_all_1D.pkl'

# Load the pickle file
with open(pkl_path, 'rb') as file:
    data = pickle.load(file)

In [5]:
train_data = data['train'].copy()
val_data = data['val'].copy()

In [6]:
train_labels = train_data['label'].copy()
temp = train_data.copy()
del temp['label']
tr_features = temp

In [7]:
val_labels = val_data['label'].copy()
temp = val_data.copy()
del temp['label']
v_features = temp

# Shuffle Data

In [8]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [9]:
train_y, train_features = shuffle_data(train_labels, tr_features)

In [10]:
val_y, val_features = shuffle_data(val_labels, v_features)

## **XGBoost Model**

In [11]:
train_results = {}
val_results = {}

val_scores = {}

In [12]:
def evaluate_model(model, validation_features, val_y):
    # Predict class labels for validation set
    val_yhat_result = model.predict(validation_features)

    # Print classification report
    print('Validation classification Report \n')
    print(classification_report(val_y, val_yhat_result))

    # Get probabilities for the validation set (for AUC calculation)
    val_y_proba = model.predict_proba(validation_features)

    # Calculate AUC for multiclass classification using 'ovr' and 'weighted' average
    auc_score = roc_auc_score(val_y, val_y_proba, multi_class='ovr', average='weighted')
    print(f'AUC Score: {auc_score}')

    # Calculate F1-score with 'weighted' average for imbalanced dataset
    f1 = f1_score(val_y, val_yhat_result, average='weighted')
    print(f'F1 Score (Weighted): {f1}')

    # Store the scores in the dictionary
    val_score = {'f1': f1, 'auc': auc_score}

    # Return the scores dictionary
    return val_score

Fit the model with training data

In [13]:
# Imma research on how to do this properly.

# I need to read the manual. They say i gotta check out the scale_pos_weight paramter if i have imbalanced data and to use AUC for evaluation
# model = xgb.XGBClassifier(booster='dart')

# param_grid = {'max_depth': [2, 4, 6],
#               "n_estimators": [50, 100, 200],
#               'learning_rate': [0.1, 0.01, 0.05],
#               'gamma': [0, 0.25, 1],
#               'scale_pos_weight': [1, 3, 5]}

# clf = GridSearchCV(
#         model,
#         param_grid,
#         verbose=0,
#         n_jobs=2,
#         cv=5,
#         scoring = 'roc_auc'
#     )

# clf.fit(training_features, train_y)
# display(clf.best_score_)
# display(clf.best_params_)



## **With Mel-Spectrogram, MFCCs, Chroma**

In [14]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['chroma'], train_features['mfcc']), axis=1)

training_features.shape

(5278, 160)

In [15]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['chroma'], val_features['mfcc']), axis=1)

validation_features.shape

(1350, 160)

In [16]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=200, random_state=1303 )
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [17]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['gbtree_200'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['gbtree_200'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.6703703703703704


## A lil break

In [19]:
model = xgb.XGBClassifier(booster='gbtree',
                          n_estimators = 100,
                          gamma = 0.25,
                          scale_pos_weight=3
                          )
model.fit(training_features, train_y)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [11:57:20] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.25, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [20]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['100_0.25gamma_scalepos3'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['100_0.25gamma_scalepos3'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.6607407407407407


## A lil break

In [23]:
model = xgb.XGBClassifier(booster='dart',
                          n_estimators = 100,
                          gamma = 0.25,
                          )
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='dart', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.25, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [24]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['dart_100_0.25gamma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['dart_100_0.25gamma'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.6607407407407407


So the gbtree booster runs faster and gives similar results.

In [25]:
model = xgb.XGBClassifier(booster='gbtree',
                          n_estimators = 50,
                          gamma = 1,
                          )
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=1, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=50, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [26]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['gbtree_50_1gamma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['gbtree_50_1gamma'] = model.score(validation_features, val_y)

Training accuracy: 0.9988632057597575
Validation accuracy: 0.6451851851851852


So the gbtree just runs fast and gives better results. Okay. Okay. With the 1gamma too. I need to try it with the NAP results.

In [27]:
model = xgb.XGBClassifier(booster='gblinear',
                          n_estimators = 100,
                          gamma = 1,
                          )
model.fit(training_features, train_y)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [12:07:39] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "gamma" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster='gblinear', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=1, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [28]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['gblinear_100_1gamma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['gblinear_100_1gamma'] = model.score(validation_features, val_y)

Training accuracy: 0.8232284956422887
Validation accuracy: 0.6555555555555556


Okay. So, the gblinear model runs faster, gives better results, and also less overfitting. Wild. Let me increase the number of estimators

In [29]:
model = xgb.XGBClassifier(booster='gblinear',
                          n_estimators = 200,
                          gamma = 1,
                          )
model.fit(training_features, train_y)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [12:08:47] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "gamma" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster='gblinear', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=1, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [30]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['gblinear_200_1gamma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['gblinear_200_1gamma'] = model.score(validation_features, val_y)

Training accuracy: 0.8474801061007957
Validation accuracy: 0.6681481481481482


Okay, so increasing the number of estimators improved the model a bit. Noted noted. And still less overfit.
Better results for precision and stuff too.


In [41]:
model = xgb.XGBClassifier(booster='gblinear',
                          n_estimators = 300,
                          gamma = 1,
                          learning_rate = 0.25
                          )
model.fit(training_features, train_y)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [12:15:37] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "gamma" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster='gblinear', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=1, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.25, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [42]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['gblinear_300_1gamma_0.25learningrate'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['gblinear_300_1gamma_0.25learningrate'] = model.score(validation_features, val_y)

Training accuracy: 0.8361121636983706
Validation accuracy: 0.6659259259259259


In [43]:
val_yhat_result = model.predict(validation_features)

print('Validation classification Report \n')
print(classification_report(val_y, val_yhat_result))

Validation classification Report 

              precision    recall  f1-score   support

           0       0.66      0.77      0.71       114
           1       0.60      0.51      0.55       141
           2       0.78      0.76      0.77       271
           3       0.87      1.00      0.93        90
           4       1.00      0.67      0.80        18
           5       0.25      0.13      0.17        15
           6       0.15      0.17      0.16        23
           7       0.98      1.00      0.99        44
           8       0.48      0.66      0.55        50
           9       0.96      0.90      0.93        49
          10       0.29      0.21      0.24        48
          11       0.43      0.33      0.37        46
          12       0.66      0.70      0.68        54
          13       0.44      0.71      0.54        48
          14       0.37      0.50      0.42        38
          15       0.49      0.36      0.41       117
          16       0.88      0.82      0.85   

Even better. Less overfit too. Increasing the learning rate worked even better too.

In [44]:
model = xgb.XGBClassifier(booster='gblinear',
                          n_estimators = 300,
                          gamma = 1,
                          learning_rate = 0.5
                          )
model.fit(training_features, train_y)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [12:18:09] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "gamma" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster='gblinear', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=1, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.5, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [45]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['gblinear_300_1gamma_0.5learningrate'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['gblinear_300_1gamma_0.5learningrate'] = model.score(validation_features, val_y)

Training accuracy: 0.854490337248958
Validation accuracy: 0.6755555555555556


In [46]:
val_yhat_result = model.predict(validation_features)

print('Validation classification Report \n')
print(classification_report(val_y, val_yhat_result))

Validation classification Report 

              precision    recall  f1-score   support

           0       0.70      0.77      0.74       114
           1       0.63      0.55      0.59       141
           2       0.79      0.77      0.78       271
           3       0.92      1.00      0.96        90
           4       1.00      0.78      0.88        18
           5       0.40      0.27      0.32        15
           6       0.17      0.22      0.19        23
           7       0.98      1.00      0.99        44
           8       0.46      0.58      0.51        50
           9       0.96      0.90      0.93        49
          10       0.38      0.27      0.32        48
          11       0.47      0.39      0.43        46
          12       0.64      0.70      0.67        54
          13       0.44      0.71      0.54        48
          14       0.33      0.55      0.42        38
          15       0.48      0.35      0.41       117
          16       0.87      0.73      0.79   

Still a good improvement

In [37]:
model = xgb.XGBClassifier(booster='gblinear',
                          n_estimators = 400,
                          gamma = 1,
                          learning_rate = 0.3
                          )
model.fit(training_features, train_y)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [12:11:31] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "gamma" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster='gblinear', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=1, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.3, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=400, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [38]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['gblinear_400_1gamma_0.3learningrate'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['gblinear_400_1gamma_0.3learningrate'] = model.score(validation_features, val_y)

Training accuracy: 0.8510799545282304
Validation accuracy: 0.6733333333333333


# Review results from all models

In [ ]:
train_results_df = pd.DataFrame(list(train_results.items()), columns=['Parameters', 'Train_Accuracy']).round(3)
val_results_df = pd.DataFrame(list(val_results.items()), columns=['Parameters', 'Val_Accuracy']).round(3)

result_df = train_results_df.merge(val_results_df, on='Parameters')
result_df = result_df.sort_values('Parameters')
result_df